In [1]:
import os
import pandas as pd
import numpy as np
import re
import unicodedata
import tabula.io as tabula

from dotenv import load_dotenv
from pathlib import Path

# Cargar variables de entorno
load_dotenv("../.env",override=True)
PDF_FILES_PATH = os.getenv("PDF_FILES_PATH")

# Lista de pdfs
pathtopdfs = os.listdir(PDF_FILES_PATH)
print(pathtopdfs)

['Global_Competitiveness_Report_2015-2016.pdf', 'TheGlobalCompetitivenessReport2016-2017_FINAL.pdf', 'TheGlobalCompetitivenessReport2017–2018.pdf', 'TheGlobalCompetitivenessReport2018.pdf', 'WEF_Annual_Report_2021_22.pdf', 'WEF_Annual_Report_2022-23.pdf', 'WEF_Annual_Report_2023_2024.pdf', 'WEF_GlobalCompetitivenessReport_2014-15.pdf', 'WEF_TheGlobalCompetitivenessReport2019.pdf', 'WEF_TheGlobalCompetitivenessReport2020.pdf']


In [2]:
pages2read = ['63-65','70-71','55','111','111','121'] # agregar páginas por pdf + periodos
pthtopdfs_lst = [os.path.join(PDF_FILES_PATH,f) for f in pathtopdfs]

tables_pdf =  []
for pdf,page in zip(pthtopdfs_lst,pages2read):
  print(pdf)
  namefile = pdf.split('/')[-1]
  periodo = re.search(r"\d+[-_]\d+|\d+",pdf.split('/')[-1])[0]
  año = re.split("[-_]",periodo)[0]
  pdftable = tabula.read_pdf(pdf, pages=page, multiple_tables=True)
  tables_pdf.append([namefile,periodo,año,pdftable])

print(tables_pdf)

/home/ariel/Escritorio/Proyectos Profesionales/1. Presentacion/Proyectos Portafolios/2. Análisis de Indices GCI/data/raw/pdf/Global_Competitiveness_Report_2015-2016.pdf


/home/ariel/Escritorio/Proyectos Profesionales/1. Presentacion/Proyectos Portafolios/2. Análisis de Indices GCI/data/raw/pdf/TheGlobalCompetitivenessReport2016-2017_FINAL.pdf
/home/ariel/Escritorio/Proyectos Profesionales/1. Presentacion/Proyectos Portafolios/2. Análisis de Indices GCI/data/raw/pdf/TheGlobalCompetitivenessReport2017–2018.pdf
/home/ariel/Escritorio/Proyectos Profesionales/1. Presentacion/Proyectos Portafolios/2. Análisis de Indices GCI/data/raw/pdf/TheGlobalCompetitivenessReport2018.pdf


feb 27, 2026 12:12:13 A. M. org.apache.pdfbox.pdmodel.font.PDType1Font <init>
ADVERTENCIA: Using fallback font LiberationSans for base font Symbol
feb 27, 2026 12:12:13 A. M. org.apache.pdfbox.pdmodel.font.PDType1Font <init>
ADVERTENCIA: Using fallback font LiberationSans for base font ZapfDingbats
feb 27, 2026 12:12:13 A. M. org.apache.pdfbox.pdmodel.font.PDType1Font <init>
ADVERTENCIA: Using fallback font LiberationSans for HelveticaNeueLTPro-Lt
feb 27, 2026 12:12:14 A. M. org.apache.pdfbox.pdmodel.font.PDType1Font <init>
ADVERTENCIA: Using fallback font LiberationSans for HelveticaNeueLTPro-Lt


/home/ariel/Escritorio/Proyectos Profesionales/1. Presentacion/Proyectos Portafolios/2. Análisis de Indices GCI/data/raw/pdf/WEF_Annual_Report_2021_22.pdf
/home/ariel/Escritorio/Proyectos Profesionales/1. Presentacion/Proyectos Portafolios/2. Análisis de Indices GCI/data/raw/pdf/WEF_Annual_Report_2022-23.pdf


java.lang.IndexOutOfBoundsException: java.lang.IndexOutOfBoundsException: Page number does not exist.

In [ ]:
def find_indx_and_descrip(indicador_string: str, patron=None):
  if not patron: patron = r"([A-Z])\w.+"
  indx = re.search(r"^\d+\.\d+",indicador_string).group()
  descrp = re.search(patron,indicador_string).group()
  return (indx, descrp)

def find_description(indice):
  indice = str(indice)
  frptron = r"^\s*(?:\d+(\.\d+)?\s+)?(.*?)\s+\.{2,}"
  ixptron = r"^\d+\.\d+"
  match_frs = re.search(frptron, indice)
  if match_frs:
    idx =  re.search(ixptron, indice).group()
    frase = match_frs.group(2)
    return (idx, frase)
  else: return indice

# Normaliza descripción de los índices
def normalizar_text(texto):
  texto = str(texto)
  # Quitar acentos
  texto = ''.join(
    c for c in unicodedata.normalize('NFD', texto)
    if unicodedata.category(c) != 'Mn'
  )
  # Todo minúsculas
  texto = texto.lower()
  # eliminar puntuación, comas, asteriscos, etc.
  texto = re.sub(r'[^a-z0-9\s]', '', texto)
  # eliminar espacios múltiples
  texto = re.sub(r'\s+', ' ', texto).strip()
  return texto


In [ ]:
# Testeos: 0:2019,1:2018,2:2017,3:2016,4:2015,5:2014
k = 4
print(tables_pdf[k][2])
table = tables_pdf[k][-1][0].T

indic_rows = pd.concat([table.iloc[0,:],table.iloc[2,:].astype(str)+' '+table.iloc[3,:]]).tolist()
#indic_rows = pd.concat([table.T.iloc[0,:],table.T.iloc[3,:]]).to_frame().T.iloc[0].tolist()
indic_rows = [indx for indx in indic_rows if bool(re.search(r"^\d+\.\d+",str(indx)))]
[find_indx_and_descrip(re.split(r"\.{3}",indx)[0]) for indx in indic_rows]


indic_rows
#indic_rows = pd.concat([tables_pdf[2][-1][0].T.iloc[0,:], tables_pdf[2][-1][0].T.iloc[4,:]]).to_frame().T.iloc[0].tolist()
#indic_rows = [indx for indx in combined_row if bool(re.search(r"^\d+\.\d+",str(indx)))]
#display(combined_row)
#indx,descrp = find_indx_and_descrip(combined_row[0])

In [37]:
# Unión de todos los años y sus respectivos índices

indicadores = []
for table in tables_pdf:
  año = table[2]
  if año in ['2016','2017']: # 2017 tiene una sola tabla
    indic_rows = pd.concat([table[-1][0].T.iloc[0,:],table[-1][0].T.iloc[4,:]]).to_frame().T.iloc[0].tolist()
    indic_rows = [indx for indx in indic_rows if bool(re.search(r"^\d+\.\d+",str(indx)))]
    indic_rows = [find_indx_and_descrip(indx)+(año,) for indx in indic_rows]
  elif año in ['2018','2019']:
    indic_rows = pd.concat(tables_pdf[0][-1]).iloc[:,0].tolist()
    indic_rows = [indx for indx in indic_rows if bool(re.search(r"^\d+\.\d+",str(indx)))]
    indic_rows = [find_indx_and_descrip(indx)+(año,) for indx in indic_rows]
  elif año in ['2014','2015']:
    table = table[-1][0].T
    indic_rows = pd.concat([table.iloc[0,:],table.iloc[2,:].astype(str)+' '+table.iloc[3,:]]).tolist()
    indic_rows = [indx for indx in indic_rows if bool(re.search(r"^\d+\.\d+",str(indx)))]
    indic_rows = [re.split(r"\.{3}",indx)[0] for indx in indic_rows]
    indic_rows = [find_indx_and_descrip(indx)+(año,) for indx in indic_rows]
  indicadores.extend(indic_rows)

indicadores = pd.DataFrame(indicadores,columns=["NUM_INDX","DESCRIPCION","AÑO"])
indicadores['DESCRIPCION_INDX_NORM'] = indicadores['DESCRIPCION'].apply(lambda x: normalizar_text(x))
#display(indicadores)

# Creación de PILAR y SUBPILAR
indicadores['PILAR'] = indicadores['NUM_INDX'].apply(lambda x: int(re.search(r'(^\d+)', x)[0]))
indicadores['SUBPILAR'] = indicadores['NUM_INDX'].apply(lambda x: int(re.search(r'(?<=\d.)(\d+)', x)[0]))

indicadores.head()

#  Creación de categorías
dic_categorias = [{"CATEGORIA_INDX":"A","CATEGORIA_DESC":"Requisitos Básicos","FACTOR":0.4, "PILAR":[1,2,3,4]},
                  {"CATEGORIA_INDX":"B","CATEGORIA_DESC":"Impulsores de Eficiencia","FACTOR":0.5, "PILAR":[5,6,7,8,9,10]},
                  {"CATEGORIA_INDX":"C","CATEGORIA_DESC":"Factores de Innovación y Sofisticación","FACTOR":0.1, "PILAR":[11,12]}]

for cat in dic_categorias:
  indicadores.loc[indicadores['PILAR'].isin(cat['PILAR']),'CATEGORIA_INDX'] = cat['CATEGORIA_INDX']
  indicadores.loc[indicadores['PILAR'].isin(cat['PILAR']),'CATEGORIA_DESC'] = cat['CATEGORIA_DESC']
  indicadores.loc[indicadores['PILAR'].isin(cat['PILAR']),'FACTOR'] = cat['FACTOR']


indicadores['AÑO'] = indicadores['AÑO'].astype(int)
indicadores = indicadores.sort_values(by=["AÑO","PILAR","SUBPILAR"]).reset_index(drop=True)

# Creación de descrpición de pilar
dic_descr_pilar = {
    1: "Instituciones", 2: "Infraestructura",
    3: "Adopción de Tecnologías de Información y Comunicación (TIC)",
    4: "Estabilidad Macroeconómica", 5: "Salud", 6: "Habilidades",
    7: "Mercado de Productos",8: "Mercado Laboral", 9: "Sistema Financiero",
    10: "Tamaño del Mercado",11: "Dinamismo Empresarial",
    12: "Capacidad de Innovación"
    }
for pilar,des in dic_descr_pilar.items():
  indicadores.loc[indicadores['PILAR']==pilar,'DESCRIPCION_PILAR'] = des


indicadores = indicadores[['NUM_INDX', 'DESCRIPCION','DESCRIPCION_INDX_NORM', 'PILAR', 'SUBPILAR','DESCRIPCION_PILAR','AÑO' ,
                          'CATEGORIA_INDX','CATEGORIA_DESC', 'FACTOR']].copy()

indicadores = indicadores.rename(columns={"DESCRIPCION":"DESCRIPCION_INDX",})
display(indicadores.head())

indicadores.to_excel("./2. Datos/bd_diccionario_indices.xlsx",index=False)


,NUM_INDX,DESCRIPCION_INDX,DESCRIPCION_INDX_NORM,PILAR,SUBPILAR,DESCRIPCION_PILAR,AÑO,CATEGORIA_INDX,CATEGORIA_DESC,FACTOR
0,1.01,Property rights,property rights,1,1,Instituciones,2014,A,Requisitos Básicos,0.4
1,1.02,Intellectual property protection,intellectual property protection,1,2,Instituciones,2014,A,Requisitos Básicos,0.4
2,1.03,Diversion of public funds,diversion of public funds,1,3,Instituciones,2014,A,Requisitos Básicos,0.4
3,1.04,Public trust in politicians,public trust in politicians,1,4,Instituciones,2014,A,Requisitos Básicos,0.4
4,1.05,Irregular payments and bribes,irregular payments and bribes,1,5,Instituciones,2014,A,Requisitos Básicos,0.4


In [ ]:
#TESTEOS

import re

# por cada test, se crea un diccionario de índices.
# OPCIONAL
pthtopdf = "/content/drive/MyDrive/Colab Notebooks/Magister - TFM/4. Informes de competitividad/WEF_GlobalCompetitivenessReport_2014-15.pdf" # cread lista de pdfs
tst = tabula.read_pdf(pthtopdf, pages='155', multiple_tables=True)

def find_description(indice):
  indice = str(indice)
  frptron = r"^\s*(?:\d+(\.\d+)?\s+)?(.*?)\s+\.{2,}"
  ixptron = r"^\d+\.\d+"
  match_frs = re.search(frptron, indice)
  if match_frs:
    idx =  re.search(ixptron, indice).group()
    frase = match_frs.group(2)
    return (idx, frase)
  else: return indice

#find_description('1.17 Ethical behavior of firms ......................................... 3.4 ..........122')


indx_first_col = tst[0].iloc[:,0].values
indx_second_col = [f"{str(x)} {str(y)}" for x,y in zip(tst[0].iloc[:,2].values, tst[0].iloc[:,3].values)]
indx_names = list(indx_first_col) + list(indx_second_col)

indx_names_transformed = [find_description(x) for x in indx_names]
indx_names_transformed = [x for x in indx_names_transformed if x not in indx_names]

df_indic = pd.DataFrame(indx_names_transformed)
#df_indic.to_excel("./diccionario_indices_aux.xlsx")
#indx_names_transformed
#[find_description(x) for x in tst[0].iloc[:,3].values]

[('1.01', 'Property rights'),
 ('1.02', 'Intellectual property protection'),
 ('1.03', 'Diversion of public funds'),
 ('1.04', 'Public trust in politicians'),
 ('1.05', 'Irregular payments and bribes'),
 ('1.06', 'Judicial independence'),
 ('1.07', 'Favoritism in decisions of government officials'),
 ('1.08', 'Wastefulness of government spending'),
 ('1.09', 'Burden of government regulation'),
 ('1.10', 'Efficiency of legal framework in settling disputes'),
 ('1.11', 'Efficiency of legal framework in challenging regs.'),
 ('1.12', 'Transparency of government policymaking'),
 ('1.13', 'Business costs of terrorism'),
 ('1.14', 'Business costs of crime and violence..................... 3.8'),
 ('1.15', 'Organized crime'),
 ('1.16', 'Reliability of police services'),
 ('1.17', 'Ethical behavior of firms'),
 ('1.18', 'Strength of auditing and reporting standards'),
 ('1.19', 'Efficacy of corporate boards'),
 ('1.20', 'Protection of minority shareholders’ interests'),
 ('1.21', 'Strength of 